In [39]:
import functools
from dataclasses import dataclass
import random
import numpy as np
from tqdm.auto import tqdm
from icecream import ic
import pandas as pd
from itertools import permutations
import pandas as pd
from geopy.distance import geodesic

In [40]:
data = pd.read_csv('cities/italy.csv', header=None , names=['City', 'x', 'y'])
# cities #to print the data of all the cities
cities = data[['x', 'y']].values
city_name = data['City'].values

In [41]:
distance = {}

for i, row1 in data.iterrows():
    country1 = row1['City']
    coord1 = (row1['x'], row1['y'])

    for j, row2 in data.iterrows():
        if i>=j:
            continue

        country2 = row2['City']
        coord2 = (row2['x'], row2['y'])

        distance_km = geodesic(coord1, coord2).kilometers

        distance[(country1, country2)] = distance_km



In [42]:
distance_df = pd.DataFrame(list(distance.items()), columns=['Country Pair', 'Distance (km)'])
print(distance_df)
distance_df.to_csv('output.csv', index = False) #used to store the data when the output is very large

           Country Pair  Distance (km)
0      (Ancona, Andria)     349.304011
1        (Ancona, Bari)     391.041868
2     (Ancona, Bergamo)     383.020439
3     (Ancona, Bologna)     199.899630
4     (Ancona, Bolzano)     364.044781
...                 ...            ...
1030    (Turin, Verona)     262.842930
1031   (Turin, Vicenza)     307.118791
1032   (Venice, Verona)     104.856922
1033  (Venice, Vicenza)      63.179244
1034  (Verona, Vicenza)      44.695173

[1035 rows x 2 columns]


In [43]:
#for global optimal distance
#to find the global optimal solution have to find the shortest path that visits each city exactly once and returns to the starting point
#trying with bruth force approch but its not optimal for huge size of the dataset

country_coords = {row['City']: (row['x'], row['y']) for _, row in data.iterrows()}

#function used to calculate the total distance of a given tour
def calculate_total_distance(tour, country_coords, countries):
    total_distance = 0
    n = len(tour)
    for i in range(n - 1):
        city1 = countries[tour[i]]
        city2 = countries[tour[i + 1]]
        total_distance += geodesic(country_coords[city1], country_coords[city2]).kilometers
    total_distance += geodesic(country_coords[countries[tour[-1]]], country_coords[countries[tour[0]]]).kilometers
    return total_distance

In [28]:
#bruth force code
def tsp_bruteforce(country_coords):
    countries = list(country_coords.keys())
    min_distance = float('inf')
    optimal_tour = None

    # Iterate through all possible permutations (tours) of the countries
    for perm in permutations(countries):
        current_distance = calculate_total_distance(perm, country_coords)
        
        # Update if a shorter tour is found
        if current_distance < min_distance:
            min_distance = current_distance
            optimal_tour = perm
    
    return optimal_tour, min_distance

In [22]:
optimal_tour, optimal_distance = tsp_bruteforce(country_coords)

print("Optimal tour (global minimum solution):", optimal_tour)
print("Total distance of optimal tour (km):", optimal_distance)

KeyboardInterrupt: 

# Genetic Algorithm

In [44]:
#first creatation of the intial population
def create_initial_population(pop_size, num_cities):
    return [random.sample(range(num_cities), num_cities) for _ in range(pop_size)]

#the funtion which can be used for the selection
def selection(population, fitness, num_parents):
    fitness_sorted = sorted(zip(population, fitness), key=lambda x: x[1])
    selected = [individual for individual, _ in fitness_sorted[:num_parents]]
    return selected

#for the crossover
def crossover(p1,p2):
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[start:end] = p1[start:end]
    pointer = end
    for gene in p2:
        if gene not in child:
            if pointer >=size:
                pointer = 0
            child[pointer] = gene
            pointer += 1
    return child

#swapping of children using mutation
def mutation(tour, mutation_rate = 0.01):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
    return tour


In [45]:
def genetic_algorithm(countries, country_coords, population_size=100, generations=500, mutation_rate=0.01, num_parents=20):
    num_cities = len(countries)
    population = create_initial_population(population_size, num_cities)

    for generation in range(generations):
        # Calculate fitness of each individual
        fitness = [calculate_total_distance(tour, country_coords, countries) for tour in population]
        
        # Selection
        parents = selection(population, fitness, num_parents)
        
        # Generate new population through crossover and mutation
        next_population = []
        while len(next_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutation(child, mutation_rate)
            next_population.append(child)
        
        population = next_population

        # Print best solution every 50 generations
        if generation % 50 == 0:
            best_distance = min(fitness)
            print(f"Generation {generation}, Best Distance: {best_distance:.2f} km")
    
    
    final_fitness = [calculate_total_distance(tour, country_coords, countries) for tour in population]
    best_tour_index = np.argmin(final_fitness)
    best_tour = population[best_tour_index]
    best_distance = final_fitness[best_tour_index]

    # Convert indices back to country names
    best_tour_countries = [countries[i] for i in best_tour]
    return best_tour_countries, best_distance

In [46]:
# Run the genetic algorithm
best_tour, best_distance = genetic_algorithm(city_name, country_coords)
print("\nOptimal Tour:", best_tour)
print("Optimal Distance (km):", best_distance)

Generation 0, Best Distance: 15628.72 km
Generation 50, Best Distance: 6567.98 km
Generation 100, Best Distance: 5742.91 km
Generation 150, Best Distance: 5640.86 km
Generation 200, Best Distance: 5499.30 km
Generation 250, Best Distance: 5148.73 km
Generation 300, Best Distance: 4832.29 km
Generation 350, Best Distance: 4823.06 km
Generation 400, Best Distance: 4823.06 km
Generation 450, Best Distance: 4732.59 km

Optimal Tour: ['Terni', 'Perugia', 'Florence', 'Leghorn', 'Prato', 'Modena', "Reggio nell'Emilia", 'Parma', 'Genoa', 'Turin', 'Novara', 'Piacenza', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Trento', 'Bolzano', 'Padua', 'Venice', 'Trieste', 'Vicenza', 'Verona', 'Ferrara', 'Bologna', 'Forlì', 'Ravenna', 'Rimini', 'Ancona', 'Pescara', 'Rome', 'Sassari', 'Cagliari', 'Palermo', 'Catania', 'Syracuse', 'Reggio di Calabria', 'Messina', 'Salerno', 'Bari', 'Taranto', 'Andria', 'Foggia', 'Giugliano in Campania', 'Naples', 'Latina']
Optimal Distance (km): 4732.5860551954565


In [3]:
# the distance can be an euclidean distance
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
#to calculate total tour distance

def create_distance_matrix(cities):
    n = len(cities)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = distance(cities[i], cities[j])
    return dist_matrix

def tot_distance(tour, cities):
    dist = 0
    for i in range(len(tour)):
        dist += distance(cities[tour[i]], cities[tour[(i+1)%len(tour)]])
    return dist


In [4]:
#Brute forse algorithm
#tries every possible permutaion of the cities to find the shortest path. 
# Slow but accurate algoritm
#permutation uses all the possible combination of the citites
def calculate_tour_cost(tour, dist_matrix):
    cost = 0
    n = len(tour)
    for i in range(n - 1):
        cost += dist_matrix[tour[i]][tour[i + 1]]
    cost += dist_matrix[tour[-1]][tour[0]]  # Return to the origin
    return cost

def tsp_bruteforce(cities, city_names):
    n = len(cities)
    min_cost = float('inf')
    best_tour = None
    steps = 0

    dist_matrix = create_distance_matrix(cities)

    # Generate all permutations of city indices
    for perm in permutations(range(n)):
        steps += n
        cost = calculate_tour_cost(perm, dist_matrix)
        # # Calculate the total cost of the current permutation (tour)
        # cost = 0
        # for i in range(n - 1):
        #     cost += distance(cities[perm[i]], cities[perm[i + 1]])
        # cost += distance(cities[perm[-1]], cities[perm[0]])  # Return to the origin
        
        # Check if this tour is better
        if cost < min_cost:
            min_cost = cost
            best_tour = perm
    
    # Convert best_tour indices to city names
    best_tour_names = [city_names[i] for i in best_tour]
    
    return best_tour, best_tour_names, min_cost, steps


In [5]:
best_tour_indices, best_tour_names, min_cost, steps = tsp_bruteforce(cities, city_name)

print(f"Brute force TSP cost: {min_cost}")
print(f"Best tour (indices): {best_tour_indices}")
print(f"Best tour (names): {best_tour_names}")
print("Tofal steps are:- ", steps)

Brute force TSP cost: 12.22271518168697
Best tour (indices): (4, 1, 7, 0, 2, 6, 5, 3)
Best tour (names): ['Norsup', 'Lakatoro', 'Vila', 'Isangel', 'Longana', 'Sola', 'Port Olry', 'Luganville']
Tofal steps are:-  322560


In [43]:
#faster but does not gurantee the shortes path
def tsp_nearest_neighbor(cities):
    n = len(cities)
    visited = [False] * n
    current_city = 0
    visited[current_city] = True
    tour = [current_city]
    total_cost = 0
    
    for _ in range(n - 1):
        next_city = None
        min_dist = float('inf')
        
        # Find the nearest unvisited city
        for i in range(n):
            if not visited[i]:
                dist = distance(cities[current_city], cities[i])
                if dist < min_dist:
                    min_dist = dist
                    next_city = i
        
        # Move to the nearest city
        tour.append(next_city)
        total_cost += min_dist
        visited[next_city] = True
        current_city = next_city
    
    # Return to the start city
    total_cost += distance(cities[current_city], cities[tour[0]])
    tour.append(tour[0])
    
    return tour, total_cost

tour_nn, cost_nn = tsp_nearest_neighbor(cities)
print(f"Nearest Neighbor TSP cost: {cost_nn}, Tour: {tour_nn}")
#  note:- maybe cost can be reduced


Nearest Neighbor TSP cost: 13.42256271273651, Tour: [0, 7, 1, 4, 3, 5, 2, 6, 0]
